# Adding robust image geometry feature

Currently donuts image geometry isn't robust and fails for certain CCD sizes. 
Here we prototype some robust geometry selection.

Reported issue:
```Dear James, I'm using Donuts to implement a guiding procedure for a small telescope, occasionally I get an error at Donuts initialization:

 File "/home/lfini/tmp/Donuts/donuts/image.py", line 172, in remove_background
    self.backsub_region = self.raw_region - self.sky_background
ValueError: operands could not be broadcast together with shapes (2928,2928) (2912,2912)

This happens with the following call:

don = donuts.Donuts(refimage=image, image_ext=0, # overscan_width=20, prescan_width=20,
                    border=64, normalise=True, exposure='EXPOSURE', subtract_bkg=True, ntiles=32)

on an Image 3056x3056 pixels wide

I can avoid the error if I uncomment the overscan and prescan arguments, setting both equal to 20.

Looking at the code I believe it's a problem depending on the fact that the image size, after removing the border, is not an integral muiltiple of number of tiles (in my case 3056-128 = 2928i with ntiles=32)

My question is:  is Donuts supposed to cope with different combinations of image sizes/border/prescan/overscan values and I've discovered a bug, or its me who must compute proper values of the arguments depending on the image size? And in the latter case how are them to be computed?

Many thanks,
                                Luca
```

Proposed solution:
   1. Rework the image geometry code to accept images of all shapes
   1. Do as requested (using available parameters)
   1. Or at least fall back to a sensible approximation of the request if the geometry requested doesn't work. 
   
The problem comes from the fact that the image background subtraction needs the image to be equally divisible into a series of tiles. This error can be seen above. On my limited testing this feature worked fine, but is clearly falling over under some circumstances. 


Current steps for trimming an image:
   1. Apply a pre/overscan correction, if required
   1. "Check if the CCD is a funny shape" - this bit seems very odd and is likely the cause of our problems
   1. Apply the border calculation to what is left of the previous step
   1. Image normalisation, if required
   1. Background subtraction, if required
   
What I'd like to happen:
   1. Apply the pre/overscan correction, if required
   1. Apply the border trimming
   1. SANITY CHECK THE REMAINING IMAGE GEOMETRY AND THE NUMBER OF TILES REQUESTED WORK TOGETHER
      1. IF NOT, TWEAK SLIGHTLY THE IMAGE GEOMETRY TO SLICE AN INTEGER NUMBER OF TILES IN BOTH AXES
   1. Image normalisation, if required
   1. Background subtraction, if required
   
There was an issue/PR submitted a long time ago that requested handling user supplied image sub-sections. Given donuts has been living in the wild for a while now I don't want to introduce non-backwards compatible changes. Therefore the handle this feature request we can do the following:
   1. Add a new switch to donuts to call for manually supplied geometry in the form of a list/tuple
   1. If this list is supplied we then:
      1. Skip pre/overscan correction
      1. Skip the border correction
      1. Continue from application of normalisation, if required
      1. Perform background subtractionm, if required
      
